# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [238]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [490]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy.stats import percentileofscore as rank



In [240]:
stocks=pd.read_csv('sp_500_stocks.csv')

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [241]:
from secrets1 import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [242]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data.status_code
data

{'companyName': 'Apple Inc',
 'marketcap': 2404086825668,
 'week52high': 187.12,
 'week52low': 129.91,
 'week52highSplitAdjustOnly': 183.3,
 'week52lowSplitAdjustOnly': 132.49,
 'week52change': 0.008460246999912607,
 'sharesOutstanding': 16732978055,
 'float': 0,
 'avg10Volume': 70002489,
 'avg30Volume': 84891937,
 'day200MovingAvg': 166.1,
 'day50MovingAvg': 146.08,
 'employees': 151109,
 'ttmEPS': 6.32,
 'ttmDividendRate': 0.9131946841285481,
 'dividendYield': 0.006189399907065176,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-22',
 'nextEarningsDate': '2022-07-22',
 'peRatio': 23.0704089142219,
 'beta': 1.3168154833995684,
 'maxChangePercent': 57.62458716061103,
 'year5ChangePercent': 3.349923984209084,
 'year2ChangePercent': 0.5397561277887032,
 'year1ChangePercent': 0.00407366729007044,
 'ytdChangePercent': -0.1850761153960806,
 'month6ChangePercent': -0.17234204116732937,
 'month3ChangePercent': -0.12621950637366153,
 'month1ChangePercent': 0.05724585584493577,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [243]:
data['year1ChangePercent']

0.00407366729007044

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [244]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

# use chunk to divide stocks into a list of symbol_groups n=100        
symbol_groups = list(chunks(stocks['Ticker'], 100))
# create empty list symbol_strings
symbol_strings = []
for i in range(0, len(symbol_groups)):
    # for each symbol_group, make strings inside list symbol_strings with join csv items
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
# define a list of cols
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [245]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:]:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    # now work on the indv symbols in the symbol_strings
    for symbol in symbol_string.split(','): # step thru indv symbols from split
        final_dataframe = final_dataframe.append(
            pd.Series
            (
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'NA'
                    # data[symbol]['stats']['month6ChangePercent'],
                    # data[symbol]['stats']['month3ChangePercent'],
                    # data[symbol]['stats']['mont1ChangePercent'],
 
                ],
                index=my_columns
            ),ignore_index=True
        )


/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3640646685.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3640646685.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3640646685.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3640646685.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

In [246]:
# print(data)
# print(symbol in symbol_string.split(','))
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,124.37,-0.18189,NA
1,AAL,14.92,-0.374457,NA
2,AAP,185.61,-0.132716,NA
3,AAPL,148.19,0.004179,NA
4,ABBV,157.23,0.37841,NA
...,...,...,...,...
496,YUM,119.45,-0.000095,NA
497,ZBH,104.81,-0.36232,NA
498,ZBRA,306.56,-0.485691,NA
499,ZION,50.30,-0.036291,NA


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [247]:
# modify dataframe named by output in descending order
final_dataframe.sort_values('One-Year Price Return',ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,80.61,2.374846,NA
1,147,DVN,54.27,0.994937,NA
2,355,OXY,58.41,0.990517,NA
3,298,MCK,331.80,0.778176,NA
4,89,CF,89.63,0.767598,NA
5,42,APA,33.07,0.702126,NA
6,138,DLTR,167.86,0.67465,NA
7,315,MRO,21.09,0.650023,NA
8,225,HRB,37.76,0.615153,NA
9,463,VAR,177.91,0.591058,NA


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [248]:
def portfolio_input():
    global portfolio_size
    while True:
        portfolio_size = input('Please enter the size of your portfolio.  \nType "n" to exit.')
        if portfolio_size == 'n':
            break
        else:
            try:
                float(portfolio_size)
                print(f'Your portfolio is ${portfolio_size}'+'.')
                break

                    
            except ValueError:
                print('You did not enter a number.  Please try again.')
    
portfolio_input()



Your portfolio is $1000000.


In [249]:
position_size=float(portfolio_size)/len(final_dataframe)
# print(len(final_dataframe))
# print(position_size)
for i in range(len(final_dataframe)):
    #final_dataframe.loc[i, 'Number of Shares to Buy']=position_size/(final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy']=math.floor(position_size/(final_dataframe.loc[i, 'Price']))
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,80.61,2.374846,248
1,147,DVN,54.27,0.994937,368
2,355,OXY,58.41,0.990517,342
3,298,MCK,331.80,0.778176,60
4,89,CF,89.63,0.767598,223
5,42,APA,33.07,0.702126,604
6,138,DLTR,167.86,0.67465,119
7,315,MRO,21.09,0.650023,948
8,225,HRB,37.76,0.615153,529
9,463,VAR,177.91,0.591058,112


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [464]:
hqm_columns = \
[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]
hqm_dataframe=pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        #print(symbol)
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
            ], 
            index=hqm_columns),ignore_index=True,
        )
hqm_dataframe

/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3313686131.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3313686131.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3313686131.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/3313686131.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,124.27,N/A,-0.178741,N/A,-0.164582,N/A,-0.062568,N/A,0.02408,N/A
1,AAL,14.91,N/A,-0.370834,N/A,-0.300917,N/A,-0.209954,N/A,-0.089615,N/A
2,AAP,192.17,N/A,-0.129207,N/A,-0.204223,N/A,-0.175039,N/A,0.022591,N/A
3,AAPL,146.84,N/A,0.004137,N/A,-0.172215,N/A,-0.127734,N/A,0.059199,N/A
4,ABBV,152.84,N/A,0.388022,N/A,0.144613,N/A,-0.093288,N/A,0.070669,N/A
...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.36,N/A,-0.000095,N/A,-0.102724,N/A,-0.010577,N/A,0.019231,N/A
497,ZBH,106.47,N/A,-0.358005,N/A,-0.196183,N/A,-0.185137,N/A,-0.087085,N/A
498,ZBRA,294.04,N/A,-0.481749,N/A,-0.480513,N/A,-0.286688,N/A,-0.04546,N/A
499,ZION,52.50,N/A,-0.037055,N/A,-0.295574,N/A,-0.219115,N/A,-0.046192,N/A


In [492]:
hqm_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:497]

#hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
# final_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_15173/1424525107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Value
273,LB,81.29,241,2.366002,100.0,0.84955,100.0,0.221928,99.396378,0.079286,93.360161,98.189135,19590.89
298,MCK,335.60,58,0.772656,99.396378,0.29428,98.792757,0.02626,91.348089,0.051368,86.720322,94.064386,19464.80
138,DLTR,167.94,116,0.693441,98.993964,0.21195,97.384306,0.00422,87.726358,0.06559,90.744467,93.712274,19481.04
225,HRB,38.28,512,0.610548,98.390342,0.673786,99.597586,0.399014,100.0,0.042487,82.897384,95.221328,19599.36
463,VAR,180.70,108,0.598177,98.189135,0.030038,81.891348,0.010623,88.933602,0.004955,66.39839,83.853119,19515.60
470,VRTX,304.25,64,0.496975,97.585513,0.300543,98.993964,0.046769,93.963783,0.159958,99.798793,97.585513,19472.00
383,PWR,132.74,147,0.477796,96.981891,0.201165,96.981891,-0.013885,82.293763,0.038237,81.086519,89.336016,19512.78
281,LLY,332.22,59,0.4403,96.37827,0.274897,98.591549,0.076654,95.975855,0.119551,98.993964,97.484909,19600.98
54,AZO,2233.80,8,0.390263,95.774648,0.080096,87.927565,0.000724,87.122736,0.046731,84.305835,88.782696,17870.40
4,ABBV,152.84,128,0.388022,95.573441,0.144613,94.768612,-0.093288,58.350101,0.070669,91.549296,85.060362,19563.52


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [494]:
time_periods = \
[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

# index down the rows
#print(row, time_period) # realized I needed to add .index to hqm_dataframe to skip headers
for row in hqm_dataframe.index: #[:2]
    for time_period in time_periods:
        time_period
        change_col= f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = (rank(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col] ) )


for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

hqm_dataframe

273        100.0
298    98.039216
138    96.078431
225    94.117647
463    92.156863
470    90.196078
383    88.235294
281     86.27451
54     84.313725
4      82.352941
363    80.392157
484    78.431373
107    76.470588
199    74.509804
335     72.54902
456    70.588235
229    68.627451
314    66.666667
366    64.705882
194    62.745098
94     60.784314
268    58.823529
104    56.862745
142    54.901961
362    52.941176
69     50.980392
130    49.019608
259    47.058824
426    45.098039
20     43.137255
85     41.176471
255    39.215686
112    37.254902
296    35.294118
205    33.333333
428    31.372549
256    29.411765
230     27.45098
310    25.490196
200    23.529412
219    21.568627
398    19.607843
232    17.647059
33     15.686275
81      13.72549
265    11.764706
76      9.803922
238     7.843137
474     5.882353
289     3.921569
378     1.960784
Name: One-Year Return Percentile, dtype: object
273        100.0
298    94.117647
138    88.235294
225    98.039216
463     27.45098


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Value
273,LB,81.29,241,2.366002,100.0,0.84955,100.0,0.221928,96.078431,0.079286,62.745098,98.189135,19590.89
298,MCK,335.60,58,0.772656,98.039216,0.29428,94.117647,0.02626,50.980392,0.051368,39.215686,94.064386,19464.80
138,DLTR,167.94,116,0.693441,96.078431,0.21195,88.235294,0.00422,37.254902,0.06559,52.941176,93.712274,19481.04
225,HRB,38.28,512,0.610548,94.117647,0.673786,98.039216,0.399014,100.0,0.042487,29.411765,95.221328,19599.36
463,VAR,180.70,108,0.598177,92.156863,0.030038,27.45098,0.010623,41.176471,0.004955,3.921569,83.853119,19515.60
470,VRTX,304.25,64,0.496975,90.196078,0.300543,96.078431,0.046769,60.784314,0.159958,98.039216,97.585513,19472.00
383,PWR,132.74,147,0.477796,88.235294,0.201165,84.313725,-0.013885,17.647059,0.038237,27.45098,89.336016,19512.78
281,LLY,332.22,59,0.4403,86.27451,0.274897,92.156863,0.076654,74.509804,0.119551,90.196078,97.484909,19600.98
54,AZO,2233.80,8,0.390263,84.313725,0.080096,35.294118,0.000724,33.333333,0.046731,31.372549,88.782696,17870.40
4,ABBV,152.84,128,0.388022,82.352941,0.144613,70.588235,-0.093288,1.960784,0.070669,56.862745,85.060362,19563.52


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [495]:
from statistics import mean

for row in hqm_dataframe.index:
    # create an empty list for each row
    momentum_percentiles = []
    for time_period in time_periods:
        # append all the return percentile values to the above list
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    # find the avg of the values inside the row's list of percentiles
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Value
273,LB,81.29,241,2.366002,100.0,0.84955,100.0,0.221928,96.078431,0.079286,62.745098,89.705882,19590.89
298,MCK,335.60,58,0.772656,98.039216,0.29428,94.117647,0.02626,50.980392,0.051368,39.215686,70.588235,19464.80
138,DLTR,167.94,116,0.693441,96.078431,0.21195,88.235294,0.00422,37.254902,0.06559,52.941176,68.627451,19481.04
225,HRB,38.28,512,0.610548,94.117647,0.673786,98.039216,0.399014,100.0,0.042487,29.411765,80.392157,19599.36
463,VAR,180.70,108,0.598177,92.156863,0.030038,27.45098,0.010623,41.176471,0.004955,3.921569,41.176471,19515.60
470,VRTX,304.25,64,0.496975,90.196078,0.300543,96.078431,0.046769,60.784314,0.159958,98.039216,86.274510,19472.00
383,PWR,132.74,147,0.477796,88.235294,0.201165,84.313725,-0.013885,17.647059,0.038237,27.45098,54.411765,19512.78
281,LLY,332.22,59,0.4403,86.27451,0.274897,92.156863,0.076654,74.509804,0.119551,90.196078,85.784314,19600.98
54,AZO,2233.80,8,0.390263,84.313725,0.080096,35.294118,0.000724,33.333333,0.046731,31.372549,46.078431,17870.40
4,ABBV,152.84,128,0.388022,82.352941,0.144613,70.588235,-0.093288,1.960784,0.070669,56.862745,52.941176,19563.52


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [496]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:51]
final_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe
df = hqm_dataframe

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [391]:
portfolio_input()

Your portfolio is $1000000.


In [500]:
def position_size(dataframe):
    print('length of dataframe:', len(dataframe))
    position_size=float(portfolio_size)/len(dataframe)
    print('position_size:', position_size)
    
    for row in dataframe.index:
        # find the avg of the values inside the row's list of percentiles
        dataframe.loc[row, 'Number of Shares to Buy']=math.floor(position_size/(dataframe.loc[row, 'Price']))
        dataframe.loc[row, 'Value'] = dataframe.loc[row, 'Price']*dataframe.loc[row, 'Number of Shares to Buy']
    # find total expenditure
    total = dataframe['Value'].sum()
    print(total)
    return dataframe

In [501]:
position_size(hqm_dataframe)

length of dataframe: 51
position_size: 19607.843137254902
994121.7400000001


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Value
273,LB,81.29,241,2.366002,100.0,0.84955,100.0,0.221928,96.078431,0.079286,62.745098,89.705882,19590.89
107,COG,22.66,865,0.346101,76.470588,0.18631,80.392157,0.259206,98.039216,0.259282,100.0,88.725490,19600.90
470,VRTX,304.25,64,0.496975,90.196078,0.300543,96.078431,0.046769,60.784314,0.159958,98.039216,86.274510,19472.00
281,LLY,332.22,59,0.4403,86.27451,0.274897,92.156863,0.076654,74.509804,0.119551,90.196078,85.784314,19600.98
225,HRB,38.28,512,0.610548,94.117647,0.673786,98.039216,0.399014,100.0,0.042487,29.411765,80.392157,19599.36
199,GIS,79.05,248,0.319443,74.509804,0.124132,64.705882,0.085255,80.392157,0.116112,88.235294,76.960784,19604.40
94,CI,292.55,67,0.212107,60.784314,0.176343,78.431373,0.10663,84.313725,0.106201,82.352941,76.470588,19600.85
314,MRK,94.00,208,0.256519,66.666667,0.175922,76.470588,0.098604,82.352941,0.094025,72.54902,74.509804,19552.00
298,MCK,335.60,58,0.772656,98.039216,0.29428,94.117647,0.02626,50.980392,0.051368,39.215686,70.588235,19464.80
230,HUM,491.92,39,0.077904,27.45098,0.259322,90.196078,0.082499,78.431373,0.102273,80.392157,69.117647,19184.88


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [255]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: